In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import xrange
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

In [2]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

## Create the Conv Net computation graph:
------------

In [3]:
imageSize = 28
numChannels = 3
numLabels = 2
batchSize = 20
numEpochs = 3
trainSize = 100


# declare graph operations
reset_graph()
# Inplementing Convolution layer
# Declare Adaptive learning rate
starter_learning_rate = 0.01
globalStep = tf.Variable(0, dtype=tf.float32)
learningRate = tf.train.exponential_decay(learning_rate = starter_learning_rate, 
                                          global_step = globalStep*batchSize, 
                                          decay_steps = trainSize, 
                                          decay_rate=0.95, 
                                          staircase=True)

# Let us add the learning Rate to the Tensorflow summary
tf.summary.scalar('Learning_Rate',learningRate)

def convActivation(inputData, inpOutShape=[5,5,3,12], name="convLayer"):
    kernelY, kernelX, numImp, numOut = inpOutShape
    with tf.name_scope(name):
        with tf.variable_scope(name):
            w = tf.get_variable(dtype=tf.float32, 
                                shape=inpOutShape, 
                                initializer=tf.random_normal_initializer(
                                    mean=0, stddev=0.1, seed = 9743),
                                name="convWghts")
            b = tf.get_variable(dtype=tf.float32,
                               shape=[numOut],
                               initializer=tf.constant_initializer(1.0),
                               name="convBias")

            conv = tf.nn.conv2d(inputData, w, strides=[1,1,1,1], padding="SAME")
            act = tf.nn.relu(conv + b)
            pool = tf.nn.max_pool(act, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
            
            # Load summaries into Tensorboard
            tf.summary.histogram("convWeights", w)
            tf.summary.histogram("convBiases", b)
            tf.summary.histogram('reluActivation', act)
            return pool
    
# Implement fully connected layer
def fcLayer(inputData, inpOutShape, name="fcLayer"):
    numInp, numOut = inpOutShape
    with tf.name_scope(name):
        with tf.variable_scope(name):
            w = tf.get_variable(dtype=tf.float32,
                               shape=inpOutShape,
                               initializer=tf.random_normal_initializer(
                                   mean=0, stddev=0.1, seed=2349),
                               name="fcWghts")
            b = tf.get_variable(dtype=tf.float32,
                               shape=[numOut],
                               initializer=tf.constant_initializer(1.0),
                               name="fcBias")
            convToFc1 = tf.matmul(inputData, w) + b

            tf.summary.histogram('fcWeights', w)
            tf.summary.histogram('fcBiases', b)
            return tf.nn.relu(convToFc1)
    
# Build the forward feed network

x = tf.placeholder(tf.float32, shape=[None, imageSize, imageSize, numChannels], name='xInp')
y = tf.placeholder(tf.float32, shape=[None, numLabels], name="xLabels")
print (x.get_shape().as_list(), y.get_shape().as_list())
conv1 = convActivation(inputData=x,
                      inpOutShape=[5,5,3,12],
                      name = "conv1")
conv2 = convActivation(inputData=conv1,
                      inpOutShape=[5,5,12,12],
                      name = "conv2")

shapeY, shapeX, depth = conv2.get_shape().as_list()[1:4]
flattenedShape = shapeY * shapeX * depth
conv2Flattened = tf.reshape(conv2, [-1, flattenedShape])
fcOut1 = fcLayer(inputData=conv2Flattened,
                inpOutShape=[flattenedShape, 28],
                name="fcLayer1")
fcOut2 = fcLayer(inputData=fcOut1,
                inpOutShape=[28,numLabels],
                name="fcLayer2")

# Now we send our input to the softmax function to calculate the cross entropy loss
with tf.name_scope("trainLoss"):
    loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits=fcOut2, labels=y)
    )
    # Add loss to the TensorBoard summary
    tf.summary.scalar("Loss",loss)

# Implement the optimizer
with tf.name_scope("trainOpt"):
    optimizer = tf.train.AdamOptimizer(learningRate).minimize(loss, global_step=globalStep)

# finally we calculate the accuracy
with tf.name_scope("Accuracy"):
    correct_prediction = tf.equal(tf.argmax(fcOut2,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # Add the accuracy to the summary
    tf.summary.scalar("Accuracy",accuracy)

[None, 28, 28, 3] [None, 2]


In [4]:
# [op for op in tf.get_default_graph().get_operations()]

### Create Session and Train:
----------

In [5]:
summaryOutput = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/TensorFlowOPS/"

def summaryBuilder(sess, outFilePath):
    mergedSummary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(outFilePath)
    writer.add_graph(sess.graph)
    return mergedSummary, writer
    
# Here we build fake mnist data
def fake_data(trainSize, numFeatures):
    """Generate a fake dataset that matches the dimensions of MNIST."""
    data = np.random.rand(
        trainSize, imageSize, imageSize, 3)
    labels = np.zeros(shape=(trainSize,), dtype=np.int64)
    for image in xrange(trainSize):
        label = image % 2
#         data[image, 0] = label - 0.5
        labels[image] = label
    return data, labels

trainData, trainLabels = fake_data(trainSize=trainSize, numFeatures=3)

trainLabels = np.eye(numLabels)[trainLabels]
numSteps = int((numEpochs * trainSize)) // batchSize
# Capture the remainder if in case the trainSize is not ezactly divisible by batchSize.  If such is the case then we may miss some training set. 
# For example, if trainingSize = 11 and batchSize=2 them since 10%2 = 0, but 11%2 = 1. So when we itereate over a batch of 2 then for every epoch 
# we miss one trainig example
remainder = trainSize % batchSize  
sess = tf.Session()
with sess.as_default():
    sess.run(tf.global_variables_initializer())
#     print ('Total number of Steps are: ', numSteps)
    # Now that the computation graph is initialized, we would want to merge all the summaties
    # to display in the tensor board.
    mergedSummary, writer = summaryBuilder(sess, summaryOutput+"/newFile")
    for step in np.arange(numSteps):
        offset = (step * batchSize) % (trainSize-remainder)
#         print ('(Step * batchSize) modulus (trainSize - remainder) = offset (%s * %s) modulus (%s - %s) = %s : '%(str(step), str(batchSize), str(trainSize), str(remainder), str(offset)))
        if offset == (trainSize - remainder - batchSize):
            batchData = trainData[offset:(offset+batchSize+remainder),:]
            batchLabels = trainLabels[offset:(offset+batchSize+remainder),:]
        else:
            batchData = trainData[offset:(offset+batchSize),:]
            batchLabels = trainLabels[offset:(offset+batchSize),:]
            
        feedDict = {x:batchData, y:batchLabels}
        
        # Before we run the batch we would like to capture and store summaries into disk
        # But we dont do it for all the batches because we dont want to overload Tensorboard with the 
        # statistics from all the steps.
        if (step%2) == 0:
            smry = sess.run(mergedSummary, feed_dict = feedDict)
            writer.add_summary(smry, step)
#         print (batchData.shape)
        
        # Now we run the optimizer 
        fc_out, lessCE, acc, gs, lr, _ = sess.run([fcOut2, loss, accuracy,
                                                        globalStep, learningRate, 
                                                        optimizer],feed_dict=feedDict)
#         print ('Accuracy : ', acc)
#         print ("globalStep : ", gs)
#         print ('learningRate : ', lr)
        
    # We have many summaries (histogram, scaler and etc.), we would want to merge them all into one graph
#     merged_summary = tf.summary.merge_all()
    # Here we store the session into the Disk
    writer = tf.summary.FileWriter(summaryOutput, sess.graph)
#     writer = tf.summary.FileWriter(summaryOutput)
#     writer.add_graph(sess.graph)
    writer.close()



In [28]:
data = np.ndarray(
        shape=(20, 28, 28, 3),
        dtype=np.float32)

In [31]:
data.shape

(20, 28, 28, 3)